In [ ]:
import numpy as np
from scipy.sparse import random as sparse_random
from scipy.sparse.linalg import eigsh
import matplotlib.pyplot as plt

def generate_sparse_matrix(size, density):
    # Step 1: Assign distinct diagonal values (evenly spaced from -1 to 1)
    print(f"Generating sparse matrix with size {size} and density {density:.2%}")
    diagonal_values = np.linspace(-1, 1, size)
    print(f"Diagonal values (first 10): {diagonal_values[:10]}")

    # Step 2: Create random values for the off-diagonals
    print(f"Generating random off-diagonal values with density {density}")
    rvs = np.random.randn  
    upper = sparse_random(size, size, density=density/2, data_rvs=rvs, format='coo')
    matrix_s = upper + upper.T
    matrix_s.setdiag(diagonal_values)
    print(f"Matrix generated with shape {matrix_s.shape} and nnz (non-zero elements) {matrix_s.nnz}")
    
    return matrix_s.tocsr()  

def lanczos_algorithm(matrix_s, initial_vector, expected_eigenvalue, tol=1e-6, max_iter=10000):
    m = matrix_s.shape[0]
    print(f"Starting Lanczos algorithm with matrix size {m}, expected eigenvalue {expected_eigenvalue:.6f}, and tolerance {tol}")
    
    krylov_vectors = np.zeros((m, max_iter + 1))
    tridiag_matrix = np.zeros((max_iter + 1, max_iter))
    w = initial_vector / np.linalg.norm(initial_vector)
    krylov_vectors[:, 0] = w
    print(f"Initial vector normalized (first 10 elements): {w[:10]}")
    
    for k in range(max_iter):
        print(f"\n--- Iteration {k+1} ---")
        v = matrix_s.dot(krylov_vectors[:, k])
        tridiag_matrix[k, k] = np.dot(krylov_vectors[:, k].conj(), v)
        print(f"Tridiagonal matrix diagonal element at [{k}, {k}] = {tridiag_matrix[k, k]:.6f}")
        v = v - tridiag_matrix[k, k] * krylov_vectors[:, k]
        if k > 0:
            v = v - tridiag_matrix[k, k-1] * krylov_vectors[:, k-1]
        tridiag_matrix[k + 1, k] = np.linalg.norm(v)
        print(f"Tridiagonal matrix off-diagonal element at [{k+1}, {k}] = {tridiag_matrix[k + 1, k]:.6f}")
        if tridiag_matrix[k + 1, k] > tol:
            krylov_vectors[:, k + 1] = v / tridiag_matrix[k + 1, k]
        else:
            print(f"Convergence reached at iteration {k+1}, stopping Lanczos.")
            break
        
        eigenvalues = np.linalg.eigvalsh(tridiag_matrix[:k + 1, :k + 1])
        estimated_eigenvalue = eigenvalues[0]
        print(f"Estimated eigenvalue: {estimated_eigenvalue:.6f}")

        if abs(estimated_eigenvalue - expected_eigenvalue) < tol:
            print(f"Converged after {k+1} iterations with eigenvalue {estimated_eigenvalue:.6f}")
            return k + 1
        if estimated_eigenvalue<=expected_eigenvalue:
            print(f"Early stop after {k+1} iterations with eigenvalue {estimated_eigenvalue:.6f}")
            return k + 1
    return max_iter

def evaluate_average_iterations(size, density_levels, initial_vector, num_trials=2, tol=1e-6, max_iter=10000):
    avg_iterations_per_density = []
    
    for density in density_levels:
        print(f"\n--- Evaluating Density Level: {density:.2%} ---")
        total_iterations = 0
        
        for trial in range(num_trials):
            print(f"\nTrial {trial+1} for density level {density:.2%}")
            matrix_s = generate_sparse_matrix(size, density)
            exact_eigenvalue, _ = eigsh(matrix_s, k=1, which='SA', tol=tol)
            exact_eigenvalue = exact_eigenvalue[0]
            print(f"Exact smallest eigenvalue from eigsh: {exact_eigenvalue:.6f}")
            
            num_iterations = lanczos_algorithm(matrix_s, initial_vector, exact_eigenvalue, tol, max_iter)
            print(f"Number of iterations for trial {trial+1}: {num_iterations}")
            total_iterations += num_iterations
        
        avg_iterations = total_iterations / num_trials
        avg_iterations_per_density.append(avg_iterations)
        print(f"Density: {density:.2%}, Average Lanczos Basis Size: {avg_iterations}")
    
    return avg_iterations_per_density

# Parameters
size = 10000
density_levels = [0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]
initial_vector = np.random.rand(size)  # random initial vector

# Evaluate the basis size for different sparsity levels
average_basis_sizes = evaluate_average_iterations(size, density_levels, initial_vector, num_trials=2)

# Plot the results
plt.figure(figsize=(10, 6))
plt.plot(density_levels, average_basis_sizes, marker='o', linestyle='-', color='b')
plt.xlabel('Density Level (%)')
plt.ylabel('Average Lanczos Basis Size')
plt.title('Density vs Average Lanczos Basis Size')
plt.grid(True)
plt.show()



--- Evaluating Density Level: 1.00% ---

Trial 1 for density level 1.00%
Generating sparse matrix with size 10000 and density 1.00%
Diagonal values (first 10): [-1.         -0.99979998 -0.99959996 -0.99939994 -0.99919992 -0.9989999
 -0.99879988 -0.99859986 -0.99839984 -0.99819982]
Generating random off-diagonal values with density 0.01
Matrix generated with shape (10000, 10000) and nnz (non-zero elements) 1007384
Exact smallest eigenvalue from eigsh: -20.345922
Starting Lanczos algorithm with matrix size 10000, expected eigenvalue -20.345922, and tolerance 1e-06
Initial vector normalized (first 10 elements): [0.01041762 0.01092008 0.0143442  0.00758388 0.01128866 0.00947694
 0.01084838 0.01095722 0.01693382 0.00512332]

--- Iteration 1 ---
Tridiagonal matrix diagonal element at [0, 0] = 0.044103
Tridiagonal matrix off-diagonal element at [1, 0] = 9.998282
Estimated eigenvalue: 0.044103

--- Iteration 2 ---
Tridiagonal matrix diagonal element at [1, 1] = -0.109111
Tridiagonal matrix of